<h3><b>CREATE DATA FRAMES.IPYNB</b><h3>
<p>Creates dataframes with all data and without excluded participants<p>

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import jsonlines
from functools import reduce
import statistics
import math
import os
import json
import ast
import warnings
pd.options.mode.copy_on_write = True
pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)
warnings.simplefilter(action='ignore', category=FutureWarning)

#uses various functions
from dataclean_func import *

In [28]:
#load in dataframes
    ##for example script: using a subset of pilot data to show how data cleaning works
df = pd.read_csv("csvs/blinded_data_full.csv", low_memory=False)
demographics=pd.read_csv("csvs/blinded_demography.csv", low_memory=False)
df.drop('Unnamed: 0', axis="columns", inplace=True)
demographics.drop('Unnamed: 0', axis="columns", inplace=True)
demographics.drop_duplicates(inplace=True)##remove duplicates

In [29]:
len(set(df.participant_no))

405

In [30]:
excluded_list=[] #create empty list of excluded participants

DEMOGRAPHICS

In [31]:
demographics[demographics.aligns!='True'].dropna() ##dropping those with missing data 

,prolific_age,prolific_sex,prolific_MH,time taken,self_report_gender,self_report_assigned-at-birth,self_report_age,aligns,participant_no
4,60,Male,Yes,66.133333,Male,Yes,61,False,5
11,31,Male,No,91.800000,Male,Yes,32,False,12
12,24,Male,No,86.350000,Male,Yes,20,False,13
34,54,Female,No,48.866667,Female,Yes,57,False,35
49,63,Male,No,55.300000,Male,Yes,64,False,50
85,20,Male,Yes,128.166667,Male,Yes,24,False,86
103,65,Male,Yes,51.133333,Male,Yes,60,False,104
124,73,Female,No,55.650000,Female,Yes,75,False,125
139,42,Male,Yes,55.450000,Male,Yes,43,False,140
141,37,Female,No,100.566667,Female,Yes,38,False,142


In [32]:
##will want to check these people individually
    #included if discrepancy is due to gender not matching what is assigned at birth
    #and if age is within 1 year
to_check=list(demographics[demographics.aligns!='True'].participant_no)
excluded_dem=[]
for participant_no in to_check:
    checking=demographics[demographics.participant_no==participant_no]
    age_discrepancy=int(checking.prolific_age)-int(checking.self_report_age)
    if -1 <= age_discrepancy <=1:
        ##check gender
        if checking.prolific_sex.iloc[0]!=checking.self_report_gender.iloc[0] and checking['self_report_assigned-at-birth'].iloc[0]=='Yes':
            excluded_list.append(participant_no)
            excluded_dem.append(participant_no)
    else:
        excluded_list.append(participant_no)
        excluded_dem.append(participant_no)
len(excluded_list)

10

Digit span and diagnosis

In [33]:
#digit span and diagnosis
digit_span=make_digit_span(df)
diagnosis=make_diagnosis(df)
excluded_digit_span=[]

##exclude those with digit span of 0 
for i in list(digit_span[digit_span.digit_span==0].participant_no):
    excluded_list.append(i)
    excluded_digit_span.append(i)

dem_df=(diagnosis.merge(digit_span, on='participant_no')).merge(demographics, on='participant_no')
len(excluded_list)

10

In [34]:
#check for those without automated diagnoses
diagnosis[~diagnosis['cleaned_diagnosis'].isin(['Yes', 'No'])]

,diagnosis,participant_no,cleaned_diagnosis,depression,anxiety,adhd,autism,ptsd,eating_disorder,ocd,bpd,bipolar,fnd,historical,unspecified


In [35]:
#manually check for those where cleaned diagnosis isn't what prolific says
dem_df[dem_df['prolific_MH'] != dem_df['cleaned_diagnosis']][['diagnosis', 'cleaned_diagnosis', 'participant_no', 'prolific_MH']]

,diagnosis,cleaned_diagnosis,participant_no,prolific_MH
14,no,No,15.0,Yes
25,no,No,26.0,Yes
28,"Yes, am currently having councelling. CBT",Yes,29.0,No
39,no,No,40.0,Yes
40,anxiety,Yes,41.0,No
48,PTSD Anxiety,Yes,49.0,No
69,no,No,70.0,Yes
74,depression,Yes,75.0,No
79,,No,80.0,Yes
80,depression,Yes,81.0,No


TIME LIMIT

In [36]:
##work out time taken from jatos -- as prolific is inaccurate
time_taken=[]
for participant in set(df.participant_no):
    sub_df=df[df.participant_no==participant]
    numbers=list(sub_df.time_elapsed)
    # Extract numbers that are greater than the next number
    peaks = [num for i, num in enumerate(numbers[:-1]) if num > numbers[i + 1]]
    peaks.append(numbers[-1])
    jatos_time_taken=np.sum(peaks)/60000 
    time_taken.append({
        'participant_no': participant,
        'jatos_time_taken': jatos_time_taken
    })
demographics=dem_df.merge(pd.DataFrame(time_taken), on='participant_no', how='left') ##CHANGED

In [37]:
demographics[demographics['jatos_time_taken']>120].sort_values(by='jatos_time_taken')

,diagnosis,participant_no,cleaned_diagnosis,depression,anxiety,adhd,autism,ptsd,eating_disorder,ocd,...,digit_span,prolific_age,prolific_sex,prolific_MH,time taken,self_report_gender,self_report_assigned-at-birth,self_report_age,aligns,jatos_time_taken
338,No,339.0,No,0,0,0,0,0,0,0,...,7.0,38,Male,No,120.700000,Male,Yes,38,True,120.435400
161,N/A,162.0,No,0,0,0,0,0,0,0,...,6.0,30,Female,No,121.133333,Female,Yes,30,True,120.714133
380,,381.0,No,0,0,0,0,0,0,0,...,7.0,66,Female,No,124.050000,Female,Yes,67,False,121.802700
259,,260.0,No,0,0,0,0,0,0,0,...,7.0,27,Male,No,124.416667,Male,Yes,27,True,123.363583
290,no,291.0,No,0,0,0,0,0,0,0,...,5.0,20,Male,No,126.216667,Male,Yes,21,False,125.471017
393,No,394.0,No,0,0,0,0,0,0,0,...,8.0,69,Male,Yes,126.266667,Male,Yes,68,False,126.065233
85,,86.0,No,0,0,0,0,0,0,0,...,8.0,20,Male,Yes,128.166667,Male,Yes,24,False,126.339900
8,,9.0,No,0,0,0,0,0,0,0,...,6.0,28,Male,No,128.033333,Male,Yes,28,True,126.626250
194,no,195.0,No,0,0,0,0,0,0,0,...,11.0,23,Female,No,128.000000,Female,Yes,23,True,127.422817
224,no,225.0,No,0,0,0,0,0,0,0,...,7.0,54,Female,Yes,128.333333,Female,Yes,54,True,127.705067


In [38]:
#exclude those who took too long
excluded_time=[]
for i in list(demographics[demographics['jatos_time_taken']>120].participant_no):
    excluded_list.append(i)
    excluded_time.append(i)
len(excluded_list)

26

VIDEO RATINGS

In [39]:
#create video ratings dataframes
total_participant=len(list(set(df.participant_no)))
ratings_df=pd.DataFrame()
chosen_stim_df=pd.DataFrame()
points_rating_df=pd.DataFrame()
for i in set(df.participant_no):     
    ##vid_ratings(df, i, "plot")
    vid_ratings_temp=vid_ratings(df, i, "rating_vids")
    ratings_df=pd.concat([ratings_df, vid_ratings_temp])
    chosen_stim_temp=vid_ratings(df, i, "chosen_stim")
    chosen_stim_df=pd.concat([chosen_stim_df, chosen_stim_temp])
    
    points_rating_temp=vid_ratings(df, i, "points_rating") 
    points_rating_df=pd.concat([points_rating_df, points_rating_temp])

chosen_stim_df=pd.concat([chosen_stim_df, points_rating_df])
chosen_stim_df=chosen_stim_df.sort_values(by='participant_no').reset_index().drop('index',axis=1)#

In [40]:
chosen_stim_df[chosen_stim_df.participant_no==2]

,Vid,trial_type,unpleasant_1,unpleasant_2,arousing_1,arousing_2,disgusting_1,disgusting_2,frightening_1,frightening_2,disgust_stim,fear_stim,participant_no,points_stim
3,0046,fear,4.0,3.0,4.0,5.0,1.0,1.0,5.0,6.0,0,1,2.0,NaN
4,points,points,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2.0,1.0
5,0888,disgust,6.0,5.0,5.0,5.0,7.0,6.0,2.0,1.0,1,0,2.0,NaN


In [41]:
##exclude any with 0 for a rating of interest
excluded_disgust=[]
excluded_fear=[]
disgust_stim_check=chosen_stim_df[chosen_stim_df.trial_type=="disgust"]
fear_stim_check=chosen_stim_df[chosen_stim_df.trial_type=="fear"]
exclude=[]
for i in set(chosen_stim_df.participant_no):
    participant_disgust=disgust_stim_check[disgust_stim_check.participant_no==i].reset_index()
    if participant_disgust.disgusting_1[0]==0:
        exclude.append(i)
        excluded_disgust.append(i)

    participant_fear=fear_stim_check[fear_stim_check.participant_no==i].reset_index()
    if participant_fear.frightening_1[0]==0:
        exclude.append(i)
        excluded_fear.append(i)

#exclude ones with a zero
for i in list(exclude):
    excluded_list.append(i)
len(excluded_list)

43

In [42]:
filtered_disgust= chosen_stim_df[chosen_stim_df['participant_no'].isin(excluded_disgust)][['participant_no', 'trial_type', 'frightening_1', 'disgusting_1']]
filtered_disgust[filtered_disgust.trial_type=='disgust']

,participant_no,trial_type,frightening_1,disgusting_1
10,4.0,disgust,0.0,0.0
236,79.0,disgust,0.0,0.0


In [43]:
filtered_fear= chosen_stim_df[chosen_stim_df['participant_no'].isin(excluded_fear)][['participant_no', 'trial_type', 'frightening_1', 'disgusting_1', 'Vid']]
filtered_fear[filtered_fear.trial_type=='fear']

,participant_no,trial_type,frightening_1,disgusting_1,Vid
95,32.0,fear,0.0,0.0,0877
134,45.0,fear,0.0,0.0,0548
205,69.0,fear,0.0,0.0,0877
213,72.0,fear,0.0,0.0,0548
232,78.0,fear,0.0,0.0,0374
237,80.0,fear,0.0,0.0,0548
320,107.0,fear,0.0,0.0,0877
504,169.0,fear,0.0,0.0,0374
657,220.0,fear,0.0,0.0,0877
773,258.0,fear,0.0,0.0,0877


In [44]:
#combine with demographic information
chosen_stim_df=pd.merge(chosen_stim_df, dem_df, on='participant_no', how='outer')

BEHAVIOURAL TASK

In [45]:
##create dataframe with all task information
complete_task_df=create_task_df(df, "no plot")

In [46]:
#exclusions on the basis of this dataframe
excluded_task=[]
task_understood=make_task_understood(df, complete_task_df, "no plot")
#task_understood[task_understood.task_understood=="No"]

for i in list(task_understood[task_understood.task_understood=="No"].participant_no):
    excluded_list.append(i)
    excluded_task.append(i)
len(excluded_list)

77

In [47]:
filtered_task=task_understood[task_understood.task_understood=="No"]
excluded_attention_check=task_understood[task_understood.attention_checks<4]
len(excluded_attention_check)

22

In [48]:
excluded_criteria=task_understood[task_understood.criteria_total==3]
excluded_criteria

,participant_no,attention_checks,long_breaks,total_time,timed_out_d,criteria_d,timed_out_f,criteria_f,timed_out_p,criteria_p,timed_out_total,criteria_total,task_understood,breaks_details
0,61.0,6,No,58.664650,1,1,1,1,1,1,3,3,No,NaN
0,114.0,5,No,50.778150,1,1,1,1,1,1,3,3,No,NaN
0,149.0,5,Yes,164.580233,1,1,1,1,1,1,3,3,No,44.58165
0,156.0,5,No,60.203100,1,1,1,1,1,1,3,3,No,NaN
0,189.0,6,No,56.885850,1,1,1,1,1,1,3,3,No,NaN
0,206.0,6,No,42.452900,1,1,1,1,1,1,3,3,No,NaN
0,214.0,6,No,44.331000,1,1,1,1,1,1,3,3,No,NaN
0,277.0,6,No,55.362817,1,1,1,1,1,1,3,3,No,NaN
0,375.0,6,No,60.759567,1,1,1,1,1,1,3,3,No,NaN


In [49]:
excluded_breaks=task_understood[task_understood.long_breaks=="Yes"]
excluded_breaks

,participant_no,attention_checks,long_breaks,total_time,timed_out_d,criteria_d,timed_out_f,criteria_f,timed_out_p,criteria_p,timed_out_total,criteria_total,task_understood,breaks_details
0,59.0,6,Yes,71.935517,0,0,1,0,0,0,1,0,No,27.96445
0,149.0,5,Yes,164.580233,1,1,1,1,1,1,3,3,No,44.58165
0,204.0,6,Yes,70.744283,1,0,1,1,1,1,3,2,No,18.55225
0,397.0,4,Yes,79.065617,0,0,0,0,0,0,0,0,No,24.65920


In [50]:
len(set(list(excluded_breaks.participant_no) + list(excluded_criteria.participant_no) + list(excluded_attention_check.participant_no)))==len(excluded_task)

True

In [51]:
#extract task outcomes - i.e., error types, error rates, win-stay/lose-shift
task_summary=make_task_outcomes(complete_task_df.drop('index', axis=1))
task_summary

,percentage_correct,mean_perseverative_er,mean_regressive_er,median_till_correct,mean_till_correct,win_stay,lose_shift,timed_out,block_no,block_type,disgust_block,fear_block,points_block,participant_no,fractals
0,0.716418,1.142857,0.428571,8.0,8.428571,0.951220,0.520000,0,1.0,Disgust,1,0,0,1.0,"['F010', 'F018']"
0,0.875000,0.000000,0.285714,6.0,6.142857,1.000000,0.733333,0,2.0,Fear,0,1,0,1.0,"['F000', 'F020']"
0,0.757143,0.714286,0.857143,10.0,9.285714,1.000000,0.739130,0,3.0,Points,0,0,1,1.0,"['F012', 'F014']"
0,0.753086,0.285714,1.714286,12.0,10.857143,0.947368,0.782609,0,2.0,Disgust,1,0,0,2.0,"['F015', 'F018']"
0,0.634615,1.000000,3.142857,13.0,13.428571,0.937500,0.641026,0,1.0,Fear,0,1,0,2.0,"['F012', 'F020']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.655629,0.571429,6.285714,18.0,20.857143,0.761364,0.580645,0,1.0,Fear,0,1,0,404.0,"['F012', 'F018']"
0,0.540000,0.250000,20.750000,42.5,47.000000,0.633929,0.551724,1,3.0,Points,0,0,1,404.0,"['F010', 'F015']"
0,0.723577,0.000000,4.000000,12.0,16.714286,0.862500,0.738095,0,3.0,Disgust,1,0,0,405.0,"['F015', 'F018']"
0,0.683333,0.714286,3.571429,13.0,15.285714,0.942029,0.780000,0,1.0,Fear,0,1,0,405.0,"['F000', 'F010']"


In [52]:
task_summary.sort_values(by='percentage_correct')

,percentage_correct,mean_perseverative_er,mean_regressive_er,median_till_correct,mean_till_correct,win_stay,lose_shift,timed_out,block_no,block_type,disgust_block,fear_block,points_block,participant_no,fractals
0,0.050000,87.000000,0.000000,97.5,97.500000,1.000000,0.006410,1,1.0,Disgust,1,0,0,210.0,"['F009', 'F018']"
0,0.070000,0.000000,0.000000,26.0,26.000000,0.962264,0.027397,1,2.0,Fear,0,1,0,210.0,"['F000', 'F020']"
0,0.245000,15.714286,5.000000,12.0,27.857143,0.906667,0.137097,1,3.0,Disgust,1,0,0,225.0,"['F000', 'F010']"
0,0.385000,5.750000,23.750000,40.5,48.250000,0.644444,0.376147,1,1.0,Disgust,1,0,0,387.0,"['F014', 'F015']"
0,0.385000,2.666667,22.666667,36.0,39.666667,0.632911,0.441667,1,1.0,Fear,0,1,0,114.0,"['F012', 'F020']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.854167,0.000000,0.142857,6.0,6.000000,0.972222,0.818182,0,3.0,Disgust,1,0,0,246.0,"['F018', 'F020']"
0,0.869565,0.142857,0.000000,6.0,5.714286,1.000000,0.888889,0,3.0,Disgust,1,0,0,386.0,"['F000', 'F009']"
0,0.872340,0.000000,0.000000,5.0,5.428571,0.871795,0.714286,0,3.0,Points,0,0,1,271.0,"['F000', 'F010']"
0,0.875000,0.000000,0.285714,6.0,6.142857,1.000000,0.733333,0,2.0,Fear,0,1,0,1.0,"['F000', 'F020']"


Add relevant video rating outcomes and demographic info to this 'task_summary' df - for hypothesis testing

In [53]:
##adding video information to task dataframe (for hypothesis testing)
stim_ratings_covariates=pd.DataFrame()
block_feedback=pd.DataFrame()
for participant_no in set(chosen_stim_df.participant_no):
    participant_df=chosen_stim_df[chosen_stim_df.participant_no==participant_no]
    disgust=participant_df[participant_df.trial_type=="disgust"]
    fear=participant_df[participant_df.trial_type=="fear"]
    valence_diff=int(fear.unpleasant_1)-int(disgust.unpleasant_1)
    arousal_diff=int(fear.arousing_1)-int(disgust.arousing_1)

    valence_habdiff=(int(fear.unpleasant_1)-int(fear.unpleasant_2))-(int(disgust.unpleasant_1)-int(disgust.unpleasant_2))
    arousal_habdiff=(int(fear.arousing_1)-int(fear.arousing_2))-(int(disgust.arousing_1)-int(disgust.arousing_2))
    
    row=pd.DataFrame({
        'participant_no': [participant_no],
        'valence_diff': [valence_diff],
        'arousal_diff': [arousal_diff],
        'valence_habdiff': [valence_habdiff],
        'arousal_habdiff': [arousal_habdiff],
    })
    stim_ratings_covariates=pd.concat([stim_ratings_covariates, row])

    block_feedback_row=pd.DataFrame({
        'participant_no': [participant_no, participant_no, participant_no],
        'block_type': ['Fear', 'Disgust', 'Points'],
        'feedback_details': [fear['Vid'].iloc[-1], disgust['Vid'].iloc[-1], 'Lose 10 Points']
        #'disgust_vid': [disgust['Vid'].iloc[-1]],
        #'fear_vid': [fear['Vid'].iloc[-1]],
        #'points_feedback': 'Lose 10 Points'
    })
    block_feedback=pd.concat([block_feedback, block_feedback_row])

In [54]:
##combine demographics, video ratings and task performance into one dataframe
task_and_ratings_summary=pd.merge(task_summary, stim_ratings_covariates, on='participant_no', how='outer')
task_and_ratings_summary=pd.merge(task_and_ratings_summary, block_feedback, on=['participant_no', 'block_type'], how='outer')
dfs = [task_and_ratings_summary, dem_df]
dem_vids_task = reduce(lambda  left,right: pd.merge(left,right,on=['participant_no'],
                                            how='outer'), dfs)

Review exclusion details

In [55]:
exclusions_df=pd.DataFrame()
for i in list(excluded_list):
    sub_df=dem_vids_task[dem_vids_task.participant_no==i]
    age=sub_df.prolific_age.iloc[0]
    if 18<=age<=24:
        age_cat='18-24'
    elif 25<=age<=34:
        age_cat='25-34'
    elif 35<=age<=44:
        age_cat='35-44'
    elif 45<=age<=54:
        age_cat='45-54'
    elif age>=55:
        age_cat='55+'
    else:
        age_cat='error'

    exclusion_reason=''
    if i in list(excluded_breaks.participant_no):
        exclusion_reason+='long breaks, '
    if i in list(excluded_criteria.participant_no):
        exclusion_reason+='reversal criteria, '
    if i in list(excluded_attention_check.participant_no):
        exclusion_reason+='task attention checks, '
    if i in excluded_time:
        exclusion_reason+='time taken, '
    if i in excluded_disgust:
        exclusion_reason+='not disgusted'
    if i in excluded_fear:
        exclusion_reason+='not fearful'

    row = ({
        'participant_no': i,
        'prolific_MH': demographics[demographics.participant_no==i].prolific_MH.iloc[0],
        'cleaned_diagnosis': sub_df[sub_df.participant_no==i].cleaned_diagnosis.iloc[0],
        'age_category': age_cat,
        'prolific_sex': demographics[demographics.participant_no==i].prolific_sex.iloc[0],
        'exclusion_reason': exclusion_reason ,
    })
    exclusions_df=pd.concat([exclusions_df, pd.DataFrame(row, index=[i])])

exclusions_df.to_csv('csvs/exclusions_df.csv')
exclusions_df

,participant_no,prolific_MH,cleaned_diagnosis,age_category,prolific_sex,exclusion_reason
13.0,13.0,No,No,18-24,Male,
35.0,35.0,No,No,45-54,Female,
86.0,86.0,Yes,No,18-24,Male,"time taken,"
104.0,104.0,Yes,Yes,55+,Male,
125.0,125.0,No,No,55+,Female,
...,...,...,...,...,...,...
343.0,343.0,No,No,18-24,Male,"task attention checks,"
346.0,346.0,Yes,Yes,25-34,Female,"task attention checks,"
375.0,375.0,Yes,No,55+,Female,"reversal criteria, not fearful"
397.0,397.0,Yes,Yes,25-34,Male,"long breaks,"


SAVE OUT as CSVs

In [56]:
##remove excluded participants
if [] in list(excluded_list):
    excluded_list.remove([])
excluded_list=(set(excluded_list))
len(excluded_list)

65

In [57]:
chosen_stim_df.to_csv('csvs/chosen_stim_df.csv')
ratings_df.to_csv('csvs/ratings_df.csv')
complete_task_df.to_csv('csvs/complete_task_df.csv')
dem_vids_task.to_csv('csvs/dem_vids_task.csv')
demographics.to_csv('csvs/demographics.csv')

In [58]:
##remove these participants and save out as csvs
excluded_chosen_stim=chosen_stim_df
excluded_ratings=ratings_df
excluded_complete_task=complete_task_df
excluded_demographics=demographics
excluded_dem_vids_task=dem_vids_task

for participant in excluded_list:
    excluded_chosen_stim=excluded_chosen_stim.loc[excluded_chosen_stim.participant_no!=participant]
    excluded_ratings=excluded_ratings.loc[excluded_ratings.participant_no!=participant]
    excluded_complete_task=excluded_complete_task.loc[excluded_complete_task.participant_no!=participant]
    excluded_demographics=excluded_demographics.loc[excluded_demographics.participant_no!=participant]
    excluded_dem_vids_task=excluded_dem_vids_task.loc[excluded_dem_vids_task.participant_no!=participant]

excluded_chosen_stim.to_csv('csvs/chosen_stim_excluded.csv')
excluded_ratings.to_csv('csvs/ratings_excluded.csv')
excluded_complete_task.to_csv('csvs/complete_task_excluded.csv')
excluded_demographics.to_csv('csvs/demographics_excluded.csv')
excluded_dem_vids_task.to_csv('csvs/dem_vids_task_excluded.csv')

<p><b>GET DEMOGRAPHIC INFORMATION</b></p>
<p>e.g., for summary in results section</p>

In [59]:
np.mean(excluded_demographics.prolific_age)

np.float64(44.82058823529412)

In [60]:
excluded_demographics['prolific_sex'].value_counts(normalize=False)

prolific_sex
Female    177
Male      163
Name: count, dtype: int64